# Wrangling Data with MongoDB
Data comes to us in lots of different ways, and one of those ways is in a database. A database is a collection of data. In that project I'm working with a MongoDB database .

## Import necessary libraries

In [1]:
from pprint import PrettyPrinter # a module that lets us see the data in an understandable way
import pandas as pd
from pymongo import MongoClient # a library for working with MongoDB databases)

In [2]:
#Construct a PrettyPrinter instance in pprint.
pp =PrettyPrinter(indent=2)

## Prepare Data

### Connect

Create a client that connects to the database running at localhost on port 27017


In [3]:
client = MongoClient(host="localhost",port=27017)

### Explore

In [4]:
#Print a list of the databases available on client
pp.pprint(list(client.list_databases()))


[ {'empty': False, 'name': 'admin', 'sizeOnDisk': 40960.0},
  {'empty': False, 'name': 'air_quality', 'sizeOnDisk': 29630464.0},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 36864.0},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 73728.0}]


In [5]:
#Assign the "air-quality" database to the variable db.
db = client['air_quality']

In [6]:
#Use the list_collections method to print a list of the collections available in db
for c in db.list_collections():
    print(c["name"])

nairobi


In [7]:
# Assign the "nairobi" collection in db to the variable name nairobi
nairobi = db["nairobi"]

In [8]:
#Use the count_documents method to see how many documents are in the nairobi collection.
nairobi.count_documents({})

755258

In [9]:
#Use the find_one method to retrieve one document from the nairobi collection
result = nairobi.find_one({})
pp.pprint(result)

{ '_id': ObjectId('64332a5399eb1d5ef3f08dde'),
  'lat': -1.3,
  'location': 29,
  'lon': 36.785,
  'sensor_id': 57,
  'sensor_type': 'SDS011',
  'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 2, 472000),
  'value': 34.43,
  'value_type': 'P2'}


In [10]:
#Use the distinct method to determine how many sensor sites are included in the nairobi collection.
nairobi.distinct("location")

[6,
 7,
 8,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 43,
 44,
 49]

In [11]:
# Determine how many readings there are for each site in the nairobi collection.
print("Documents from site 6:", nairobi.count_documents({"location":7}))
print("Documents from site 29:", nairobi.count_documents({"location":29}))

Documents from site 6: 211376
Documents from site 29: 63466


In [12]:
#Use the aggregate method to determine how many readings there are for each site in the nairobi collection.
result =nairobi.aggregate([
    {"$group" : {"_id":"$location", "count":{"$sum":1}}}
])
pp.pprint(list(result))

[ {'_id': 29, 'count': 63466},
  {'_id': 36, 'count': 1850},
  {'_id': 25, 'count': 38052},
  {'_id': 31, 'count': 53928},
  {'_id': 27, 'count': 60704},
  {'_id': 43, 'count': 894},
  {'_id': 6, 'count': 34050},
  {'_id': 49, 'count': 2},
  {'_id': 26, 'count': 27450},
  {'_id': 30, 'count': 55280},
  {'_id': 7, 'count': 211376},
  {'_id': 8, 'count': 17662},
  {'_id': 37, 'count': 40410},
  {'_id': 34, 'count': 42178},
  {'_id': 35, 'count': 26020},
  {'_id': 32, 'count': 7924},
  {'_id': 33, 'count': 8978},
  {'_id': 28, 'count': 12846},
  {'_id': 38, 'count': 30520},
  {'_id': 44, 'count': 6394},
  {'_id': 40, 'count': 7590},
  {'_id': 41, 'count': 7684}]


In [13]:
#determine how many types of measurements have been taken in the nairobi collection.
nairobi.distinct("value_type")

['P1', 'P2', 'humidity', 'temperature']

In [14]:
#retrieve the PM 2.5 readings from all sites. 
result = nairobi.find({"value_type":"P2"}).limit(3)
pp.pprint(list(result))

[ { '_id': ObjectId('64332a5399eb1d5ef3f08dde'),
    'lat': -1.3,
    'location': 29,
    'lon': 36.785,
    'sensor_id': 57,
    'sensor_type': 'SDS011',
    'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 2, 472000),
    'value': 34.43,
    'value_type': 'P2'},
  { '_id': ObjectId('64332a5399eb1d5ef3f08de2'),
    'lat': -1.298,
    'location': 7,
    'lon': 36.791,
    'sensor_id': 69,
    'sensor_type': 'SDS011',
    'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 7, 536000),
    'value': 16.63,
    'value_type': 'P2'},
  { '_id': ObjectId('64332a5399eb1d5ef3f08de7'),
    'lat': -1.298,
    'location': 7,
    'lon': 36.791,
    'sensor_id': 40,
    'sensor_type': 'SDS011',
    'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 26, 722000),
    'value': 19.3,
    'value_type': 'P2'}]


In [15]:
#Calculate how many readings there are for each type ("humidity", "temperature", "P2", and "P1") in site 7
result = nairobi.aggregate(
    [
        {"$match":{"location":7}},
        {"$group" : {"_id":"$value_type", "count":{"$sum":1}}}
    ]
)
pp.pprint(list(result))

[ {'_id': 'P1', 'count': 60719},
  {'_id': 'temperature', 'count': 44969},
  {'_id': 'P2', 'count': 60719},
  {'_id': 'humidity', 'count': 44969}]


In [18]:
#Calculate how many readings there are for each type ("humidity", "temperature", "P2", and "P1") in site 29
result = nairobi.aggregate(
    [
        {"$match":{"location":29}},
        {"$group" : {"_id":"$value_type", "count":{"$sum":1}}}
    ]
)
pp.pprint(list(result))

[ {'_id': 'P1', 'count': 15848},
  {'_id': 'humidity', 'count': 15885},
  {'_id': 'temperature', 'count': 15885},
  {'_id': 'P2', 'count': 15848}]


In [19]:
result = nairobi.find(
    {"location":7,"value_type":"P2"},
    projection={"value":1,"timestamp":1,"_id":0}
)
pp.pprint(result.next())

{'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 7, 536000), 'value': 16.63}


In [20]:
#Read records from last result into the DataFrame df
df = pd.DataFrame(result).set_index("timestamp")
df.columns = ['P2']
df.head()

,P2
timestamp,
2018-09-01 00:00:26.722,19.30
2018-09-01 00:00:43.434,16.90
2018-09-01 00:01:19.344,16.10
2018-09-01 00:01:26.343,10.83
2018-09-01 00:01:29.906,18.47


In [21]:
df.to_csv("D:\WorldQuant\Project3 air quality\data\my_data2.csv")
